In [2]:
import numpy as np

In [9]:
def precision(relevance_query: list) -> float:
    """
    Clacula la Precision para una consulta, definida como la proporción de documentos recuperados que son relevantes.

    Precisión = (Número de documentos relevantes recuperados) / (Número total de documentos recuperados)

    Args:
        relevance_query: Una lista de enteros (0 o 1) que indica la relevancia de cada documento.
                         1 significa relevante, 0 significa no relevante.
    Returns:
        El valor de precisión para la consulta dada.
    """
    relevance_vector = np.array(relevance_query)
    return np.sum(relevance_vector) / len(relevance_vector)

def precision_at_k(relevance_query: list, k: int) -> float:
    """
    Calcula la Precision@K (P@K) para una consulta, definida como la proporción de los primeros K documentos recuperados que son relevantes.

    Precisión@K = (Número de documentos relevantes en los primeros K documentos recuperados) / K
    Args:
        relevance_query: Una lista de enteros (0 o 1) que indica la relevancia de cada documento.
                         1 significa relevante, 0 significa no relevante.
        k: El número de documentos a considerar desde el inicio de la lista.
    Returns:
        El valor de Precision@K para la consulta dada.
    """
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    return np.sum(relevance_vector_at_k) / k if k > 0 else 0.0

def recall_at_k(relevance_query: list,
                number_relevant_docs: int,
                k: int) -> float:
    """
    Calcula la Recall@K (R@K) para una consulta, definida como la proporción de documentos relevantes recuperados entre los primeros K documentos.
    Recall@K = (Número de documentos relevantes en los primeros K documentos recuperados) / (Número total de documentos relevantes)
    Args:
        relevance_query: Una lista de enteros (0 o 1) que indica la relevancia de cada documento.
                         1 significa relevante, 0 significa no relevante.
        number_relevant_docs: El número total de documentos relevantes para la consulta.
        k: El número de documentos a considerar desde el inicio de la lista.
    Returns:
        El valor de Recall@K para la consulta dada.
    """
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    return np.sum(relevance_vector_at_k) / number_relevant_docs if number_relevant_docs > 0 else 0.0

def average_precision(relevance_query: list) -> float:
    """
    Calcula la Average Precision (AP) para una consulta dada usando NumPy.

    Args:
        relevance_query: Una lista de enteros (0 o 1) que indica la relevancia de cada documento.
                         1 significa relevante, 0 significa no relevante.

    Returns:
        El valor de Average Precision.
    """
    relevance_vector = np.array(relevance_query)
    relevant_indices = np.where(relevance_vector == 1)[0]
    
    if relevant_indices.size == 0:
        return 0.0

    precision_sum = 0.0
    for index in relevant_indices:
        precision_sum += precision_at_k(relevance_query, index + 1)

    return precision_sum / len(relevant_indices)

def mean_average_precision(vector_list):
    """
    Calcula la Mean Average Precision (MAP) para una lista de consultas en recuperación ranqueada.

    MAP evalúa la precisión promedio considerando todos los puntos en el ranking donde se recupera un documento relevante,
    sin necesidad de fijar un valor específico de K. Para cada consulta, se calcula la Average Precision (AP) como el promedio
    de las precisiones en cada posición donde se recupera un documento relevante. MAP es el promedio de las APs de todas las consultas.

    Pasos de cálculo:
    1. Se recorre el ranking de documentos recuperados.
    2. Cada vez que se encuentra un documento relevante, se mide la precisión en esa posición (P@K).
    3. Se promedian todos los valores de precisión obtenidos en posiciones relevantes.
    4. MAP es el promedio de las APs de todas las consultas.

    Args:
        vector_list: Lista de listas, donde cada sublista es un vector binario de relevancia (0 o 1) para una consulta.

    Returns:
        float: El valor de Mean Average Precision para el conjunto de consultas.
    """
    sum_average_precision = 0.0
    for vector in vector_list:
        sum_average_precision += average_precision(vector)

    return sum_average_precision/len(vector_list)

def dcg_at_k(relevance_query: list, k: int) -> float:
    """
    Calcula la Discounted Cumulative Gain (DCG) hasta la posición K para una consulta dada.
    DCG mide la utilidad de un documento basado en su posición en el ranking, penalizando los documentos relevantes que aparecen más abajo.
    Args:
        relevance_query: Una lista de enteros que indica la relevancia de cada documento.
                         Valores más altos indican mayor relevancia.
        k: El número de documentos a considerar desde el inicio de la lista.
        Returns:
        El valor de DCG@K para la consulta dada."""
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    dcg_at_k = 0.0
    for index, relevance in enumerate(relevance_vector_at_k):
        dcg_at_k += relevance/np.log2(max(index+1,2))
    return dcg_at_k

def ndcg_at_k(relevance_query: list, k: int) ->float:
    """
    Calcula la Normalized Discounted Cumulative Gain (NDCG) hasta la posición K para una consulta dada.
    NDCG normaliza el DCG dividiéndolo por el DCG ideal (IDCG), que es el DCG obtenido si todos los documentos relevantes
    estuvieran ordenados perfectamente al inicio del ranking.
    Args:
        relevance_query: Una lista de enteros que indica la relevancia de cada documento.
                         Valores más altos indican mayor relevancia.
        k: El número de documentos a considerar desde el inicio de la lista.
    Returns:
        El valor de NDCG@K para la consulta dada.
    """
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    dcg_at_k = 0.0
    
    for index, relevance in enumerate(relevance_vector_at_k):
        dcg_at_k += relevance/np.log2(max(index+1,2))
    
    ordered_relevance_vector = np.sort(relevance_vector)[::-1]
    ordered_relevance_vector_at_k= ordered_relevance_vector[:k]
    best_dcg_at_k = 0.0
    
    for index, relevance in enumerate(ordered_relevance_vector_at_k):
        best_dcg_at_k += relevance/np.log2(max(index+1,2))

    return dcg_at_k/best_dcg_at_k

In [10]:
if __name__ == "__main__":
    relevance_query_1 = [0, 0, 0, 1]
    print(f"Precision para {relevance_query_1}: {precision(relevance_query_1)}") # 0.25

    relevance_query_1 = [0, 0, 0, 1]
    k = 1
    print(f"Precision@k para {relevance_query_1} con k={k}: {precision_at_k(relevance_query_1, k)}") # 0

    relevance_query_1 = [0, 0, 0, 1]
    k = 1
    number_relevant_docs = 4
    print(f"Recall@k para {relevance_query_1} con k={k}: {recall_at_k(relevance_query_1, number_relevant_docs, k)}") # 0

    relevance_query_2 = [0,1,0,1,1,1,1]
    print(f"Average Precision para {relevance_query_2}: {average_precision(relevance_query_2)}") # 0.5961904

    queries = [
        [0,1,0,1,1,1,1],
        [1,0,1]
    ]
    print(f"Mean Average Precision para {queries}: {mean_average_precision(queries)}") # 0.7147619047619047

    relevance_query_3 = [4, 4, 3, 0, 0, 1, 3, 3, 3, 0]
    k = 6
    print(f"DCG@k para {relevance_query_3} con k={k}: {dcg_at_k(relevance_query_3, k)}") # 10.27964

    relevance_query_3 = [4, 4, 3, 0, 0, 1, 3, 3, 3, 0]
    k = 6
    print(f"NDCG@k para {relevance_query_3} con k={k}: {ndcg_at_k(relevance_query_3, k)}") # 0.7424

Precision para [0, 0, 0, 1]: 0.25
Precision@k para [0, 0, 0, 1] con k=1: 0.0
Recall@k para [0, 0, 0, 1] con k=1: 0.0
Average Precision para [0, 1, 0, 1, 1, 1, 1]: 0.5961904761904762
Mean Average Precision para [[0, 1, 0, 1, 1, 1, 1], [1, 0, 1]]: 0.7147619047619047
DCG@k para [4, 4, 3, 0, 0, 1, 3, 3, 3, 0] con k=6: 10.279642067948915
NDCG@k para [4, 4, 3, 0, 0, 1, 3, 3, 3, 0] con k=6: 0.7424602308163405
